In [7]:
import pandas as pd
import nltk
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

In [8]:
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(file('C:\\WinPython-64bit-2.7.10.2\\work\\TA\\big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [9]:
ds=pd.read_excel('C:\\WinPython-64bit-2.7.10.2\\work\\TA\\reviews_1.xlsx',sheetname='reviews')

In [10]:
innovation_rev=ds[ds['dr_inn']==1]
len(innovation_rev)

644

In [16]:
rev_split=[re.sub("[^a-zA-Z]", " ", review).lower().split() for review in innovation_rev['message_reviewerer']]
spell_correct=[[correct(word) for word in rev] for rev in rev_split]
rev_correct_inn=[" ".join(rev) for rev in spell_correct]

In [11]:
def itemspresent(item,sentence):
    res=True
    for itm in item.split('|'):
        if itm.lower() not in sentence.lower().split():
            res=res&False
    return res

def finditemsbypos(item,sentence,dist=10):
    ret=False
    firstpos=[i for i,x in enumerate(sentence.split()) if x.lower()==item.split('|')[0]]
    for val in firstpos:
        ret=itemspresent("|".join(item.split('|')[1:])," ".join(sentence.split()[val:val+dist]))
        if ret==True:
            break
    return ret 

In [49]:
arraylookup=['improve','dislike','problem','terrible','not enough','cons','downside','malfunction'
             ,'wish','could|better','con ','want to','would|better','like to','would like'
             ,'needs a','complaint','drawback','not satisfact','not|recommend','dont|recommend'
             ,'require']
print arraylookup

['improve', 'dislike', 'problem', 'terrible', 'not enough', 'cons', 'downside', 'malfunction', 'wish', 'could|better', 'con ', 'want to', 'would|better', 'like to', 'would like', 'needs a', 'complaint', 'drawback', 'not satisfact', 'not|recommend', 'dont|recommend', 'require']


In [54]:
arraylookup=['customizable design']
innova=[]
i=0
for rev in rev_correct_inn:
    innova.append(False)
    
    for item in arraylookup:
        if '|' in item:
            if finditemsbypos(item,rev)==True:
                innova[i]=True
                break
        else:
            if item.lower() in rev.lower():
                innova[i]=True                            
                break

    i=i+1
tagged=[]
i=0
for inn in innova:
    if inn==True:
        tagged.append(rev_correct_inn[i])
    i=i+1

print len(tagged)
print tagged


0
[]


In [13]:
ds_noninn=pd.read_excel('C:\\WinPython-64bit-2.7.10.2\\work\\TA\\reviews_1.xlsx',sheetname='non_inn')

In [51]:
len(rev_correct_noninn)

2641

In [15]:
rev_split=[re.sub("[^a-zA-Z]", " ", review).lower().split() for review in ds_noninn['message_reviewerer']]
spell_correct=[[correct(word) for word in rev] for rev in rev_split]
rev_correct_noninn=[" ".join(rev) for rev in spell_correct]

In [45]:
#arraylookup=['and purchase']
innova=[]
i=0
for rev in rev_correct_noninn:
    innova.append(False)
    
    for item in arraylookup:
        if '|' in item:
            if finditemsbypos(item,rev)==True:
                innova[i]=True
                break
        else:
            if item.lower() in rev.lower():
                innova[i]=True                            
                break
    if innova[i]==True:
        print item
        print rev

    i=i+1

tagged=[]
i=0
for inn in innova:
    if inn==True:
        tagged.append(rev_correct_noninn[i])
    i=i+1

print len(tagged)

require
i purchased this right before thanksgiving for a second home i spent lots of time searching refrigerator i was replacing an old side by side in a small kitchen i wanted the counter depth and hobby did not want the french door which we have in our primary home also did not want to spend an arm and a leg on a refrigerator the main problem with all of the figs seemed to be the ice maker the ice maker is very slow when dispersing the tubes this one was on sale and had adequate reviews looking at the online reviews even the highly rates reviews seemed to have a fair number of negative reviews the painless does seem to spot more easily than a kitchen aid that i have we were away for about weeks and even without leaving much in the fig when we first opened it there was a slight odor we removed a few things that we thought might have contributed and it seems to be of as long as it doesn require any repairs in the next few years i will be happy so far so good read less
require
i purchas